In [1]:
import entity_formatter
from entity_tagger import entity_tagger as tagger
import requests
import json
import pandas as pd
import subprocess
import boto3
import traceback
import json
import dataset_creator
import os
import pickle
from os import listdir
from os.path import isfile, join
from labeling_functions import *
import snorkel
from snorkel.labeling.model import LabelModel
from snorkel.labeling import PandasLFApplier
from dateutil.parser import parse
import re
from subprocess import Popen
import datetime
from snorkel.utils import probs_to_preds
import snorkel_pipeline

# Defining names for each entity types

In [2]:
inv_et_dct = {
 0: 'NULL',
 1: 'COMPANY NAME',
 2: 'DATE',
 3: 'PERSON NAME',
 4: 'CURRENCY',
 5: 'NUMBER',
 6: 'PERCENT',
 7: 'ROUTE',
 8: 'STATE',
 9: 'CITY',
 10: 'ZIPCODE',
 11: 'YEAR',
 12: 'OTHER',
 13: 'TIME',
 14: 'SPECIAL_NUMBER',
 15: 'LIST',
 16: 'TEXT'

}

In [3]:
lfs = [lf_contains_date_parser,lf_contains_currency,lf_contains_zipcode,\
       lf_contains_state,lf_contains_quntity,lf_contains_phonenumber,lf_contains_SSN,\
       lf_contains_first_name,lf_contains_last_name,lf_contains_last_percent]

def get_rid_list():
    onlyfiles = [f for f in listdir("rids_to_process/") if isfile(join("rids_to_process/", f))]
    onlyfiles = ["rids_to_process/" + s for s in onlyfiles]
    return onlyfiles

def rid_iterator(df):
    final_return = pd.DataFrame()
    for i,j in df.iterrows():
        just_holder = dataset_creator.aggregate_formatted_entities(j.rid)
        final_return = final_return.append(pd.DataFrame(just_holder))
    return final_return

In [4]:
def get_snorkel_labels(frame_to_train,pkl_name):
    print("==============================Labeling is now started=======================================")
    applier = PandasLFApplier(lfs=lfs)
    L_train = applier.apply(df=frame_to_train)
    date_parser_coverage, currency_coverage,\
    zipcode_coverage,state_coverage,\
    quntity_coverage,phonenumber_coverage,SSN_coverage,\
    first_name_coverage,last_name_coverage,percent_coverge= (L_train != ABSTAIN).mean(axis=0)
    frame_to_train.rename(columns={"word_id":"word_tokens","text":"ocr","label_number":"preds"},inplace=True)
    print("==============================Labeling is now complete=======================================")
    print("==============================Summary Stats==================================================")
    print(f"date_parser_coverage: {date_parser_coverage * 100:.1f}%")
    print(f"currency_coverage: {currency_coverage * 100:.1f}%")
    print(f"zipcode_coverage: {zipcode_coverage * 100:.1f}%")
    print(f"state_coverage: {state_coverage * 100:.1f}%")
    print(f"quntity_coverage: {quntity_coverage * 100:.1f}%")
    print(f"phonenumber_coverage: {phonenumber_coverage * 100:.1f}%")
    print(f"SSN_coverage: {SSN_coverage * 100:.1f}%")
    print(f"first_name_coverage: {first_name_coverage * 100:.1f}%")
    print(f"last_name_coverage: {last_name_coverage * 100:.1f}%")
    #print(f"alpha_number_coverage: {alpha_number_coverage * 100:.1f}%")
#     lol= f"{pkl_name}.pkl"
#     print("File name I got:", lol)
#     print(f"percent_coverage: {percent_coverge * 100:.1f}%")
#     with open(lol, 'rb') as f:
#         label_model = pickle.load(f)
    label_model = LabelModel(cardinality=15, verbose=True)
    label_model.fit(L_train=L_train, n_epochs=500, log_freq=100, seed=123)
    frame_to_train["label_number"] = label_model.predict(L=L_train, tie_break_policy="abstain")
    frame_to_train.label_number.fillna(0,inplace=True)
    frame_to_train['pred_names'] = frame_to_train.label_number.map(inv_et_dct)
    return frame_to_train
#dataset_df = pd.DataFrame()
    return frame_to_train

def make_frame(frame,doc_name):
    labeld_frame = pd.DataFrame()
    list_rids =  frame.rid.unique().tolist()
    for rid in list_rids:
        temp = frame[frame["rid"]==str(rid)]
        print("value of i=",rid)
        print("Length of rids:", temp.shape)
        just_holder = get_snorkel_labels(temp,doc_name)
        labeld_frame = pd.concat([labeld_frame,just_holder],axis=0)
    return labeld_frame
        
    

In [ ]:
paths = get_rid_list()
for i in paths:
    try:
        
        holder = pd.read_csv(i,names=["rid"])
        holder.drop_duplicates(inplace=True)
        holder.fillna("aa",axis=0,inplace=True)
        #holder = holder.head(5)
        print(f"===============Dataset generation is in progress for {i}===============")
        iterator = rid_iterator(holder)
        #dataset_df = dataset_df.append(iterator, ignore_index=True)
        print(f"==============================Dataset generation is now complete for {i} ======================================")
        print(f"==============================Predicting Labels is now complete for {i} ======================================")
        trained_labels = make_frame(iterator,i.split("/")[1])
        #trained_labels = get_snorkel_labels(iterator,i.split("/")[1])
        
#         print("==============================Getting OCR files that are needed======================================")
#         #proc = subprocess.Popen('./get_required_jsons.sh %s' %(i) , stdout=subprocess.PIPE, stderr=subprocess.PIPE,shell=True)
#         temp = i.split("/")[1]
#         print(temp)
#         temp = temp.replace("+","/")
#         snorkel_pipeline.start_gathering(str(temp.replace("_"," ")),trained_labels.rid.unique(),trained_labels)
#         #dataset_df = pd.DataFrame()
    except:
        print("*****************************************************************")
        print("Didn't get to process the document: ",i)
        print(traceback.format_exc())
        print("*****************************************************************")
    
    

===============Dataset generation is in progress for rids_to_process/DEMOGRAPHIC_ADDENDUM===============
Traceback (most recent call last):
  File "/home/ec2-user/SageMaker/snorkel_tagger/dataset_creator.py", line 42, in rememberv2_read
    results = json.loads(requests.post(url=url, data=json.dumps(payload), headers=headers).text)["Results"]
KeyError: 'Results'

==========================OBSERVATIONS OF SOME FALILURES(failed for rid)=======================================
Traceback (most recent call last):
  File "/home/ec2-user/SageMaker/snorkel_tagger/dataset_creator.py", line 119, in aggregate_formatted_entities
    recall_txn = rememberv2_read(docid)[0]
KeyError: 0

RID ====>  ad084576-8623-4752-9f8e-d1ae0506b5c7 
Traceback (most recent call last):
  File "/home/ec2-user/SageMaker/snorkel_tagger/dataset_creator.py", line 42, in rememberv2_read
    results = json.loads(requests.post(url=url, data=json.dumps(payload), headers=headers).text)["Results"]
KeyError: 'Results'

==========

Traceback (most recent call last):
  File "/home/ec2-user/SageMaker/snorkel_tagger/dataset_creator.py", line 42, in rememberv2_read
    results = json.loads(requests.post(url=url, data=json.dumps(payload), headers=headers).text)["Results"]
KeyError: 'Results'

==========================OBSERVATIONS OF SOME FALILURES(failed for rid)=======================================
Traceback (most recent call last):
  File "/home/ec2-user/SageMaker/snorkel_tagger/dataset_creator.py", line 119, in aggregate_formatted_entities
    recall_txn = rememberv2_read(docid)[0]
KeyError: 0

RID ====>  04551a81-4c55-4952-90ea-3829269df6d4 
Traceback (most recent call last):
  File "/home/ec2-user/SageMaker/snorkel_tagger/dataset_creator.py", line 42, in rememberv2_read
    results = json.loads(requests.post(url=url, data=json.dumps(payload), headers=headers).text)["Results"]
KeyError: 'Results'

==========================OBSERVATIONS OF SOME FALILURES(failed for rid)=======================================
Tra

Traceback (most recent call last):
  File "/home/ec2-user/SageMaker/snorkel_tagger/dataset_creator.py", line 42, in rememberv2_read
    results = json.loads(requests.post(url=url, data=json.dumps(payload), headers=headers).text)["Results"]
KeyError: 'Results'

==========================OBSERVATIONS OF SOME FALILURES(failed for rid)=======================================
Traceback (most recent call last):
  File "/home/ec2-user/SageMaker/snorkel_tagger/dataset_creator.py", line 119, in aggregate_formatted_entities
    recall_txn = rememberv2_read(docid)[0]
KeyError: 0

RID ====>  8c12d5c9-11dd-425d-98a2-cfd11edd969e  
Traceback (most recent call last):
  File "/home/ec2-user/SageMaker/snorkel_tagger/dataset_creator.py", line 42, in rememberv2_read
    results = json.loads(requests.post(url=url, data=json.dumps(payload), headers=headers).text)["Results"]
KeyError: 'Results'

==========================OBSERVATIONS OF SOME FALILURES(failed for rid)=======================================
Tr

Traceback (most recent call last):
  File "/home/ec2-user/SageMaker/snorkel_tagger/dataset_creator.py", line 42, in rememberv2_read
    results = json.loads(requests.post(url=url, data=json.dumps(payload), headers=headers).text)["Results"]
KeyError: 'Results'

==========================OBSERVATIONS OF SOME FALILURES(failed for rid)=======================================
Traceback (most recent call last):
  File "/home/ec2-user/SageMaker/snorkel_tagger/dataset_creator.py", line 119, in aggregate_formatted_entities
    recall_txn = rememberv2_read(docid)[0]
KeyError: 0

RID ====>  e22f5f6e-8dbc-4acd-ae24-25df5146eb6d  
Traceback (most recent call last):
  File "/home/ec2-user/SageMaker/snorkel_tagger/dataset_creator.py", line 42, in rememberv2_read
    results = json.loads(requests.post(url=url, data=json.dumps(payload), headers=headers).text)["Results"]
KeyError: 'Results'

==========================OBSERVATIONS OF SOME FALILURES(failed for rid)=======================================
Tr

Traceback (most recent call last):
  File "/home/ec2-user/SageMaker/snorkel_tagger/dataset_creator.py", line 42, in rememberv2_read
    results = json.loads(requests.post(url=url, data=json.dumps(payload), headers=headers).text)["Results"]
KeyError: 'Results'

==========================OBSERVATIONS OF SOME FALILURES(failed for rid)=======================================
Traceback (most recent call last):
  File "/home/ec2-user/SageMaker/snorkel_tagger/dataset_creator.py", line 119, in aggregate_formatted_entities
    recall_txn = rememberv2_read(docid)[0]
KeyError: 0

RID ====>  bf9e112d-6a3b-48a3-b0f1-daeb6bdafafa  
Traceback (most recent call last):
  File "/home/ec2-user/SageMaker/snorkel_tagger/dataset_creator.py", line 42, in rememberv2_read
    results = json.loads(requests.post(url=url, data=json.dumps(payload), headers=headers).text)["Results"]
KeyError: 'Results'

==========================OBSERVATIONS OF SOME FALILURES(failed for rid)=======================================
Tr

Traceback (most recent call last):
  File "/home/ec2-user/SageMaker/snorkel_tagger/dataset_creator.py", line 42, in rememberv2_read
    results = json.loads(requests.post(url=url, data=json.dumps(payload), headers=headers).text)["Results"]
KeyError: 'Results'

==========================OBSERVATIONS OF SOME FALILURES(failed for rid)=======================================
Traceback (most recent call last):
  File "/home/ec2-user/SageMaker/snorkel_tagger/dataset_creator.py", line 119, in aggregate_formatted_entities
    recall_txn = rememberv2_read(docid)[0]
KeyError: 0

RID ====>  7e384eeb-1caa-43ea-83d6-28fd713668e0  
Traceback (most recent call last):
  File "/home/ec2-user/SageMaker/snorkel_tagger/dataset_creator.py", line 42, in rememberv2_read
    results = json.loads(requests.post(url=url, data=json.dumps(payload), headers=headers).text)["Results"]
KeyError: 'Results'

==========================OBSERVATIONS OF SOME FALILURES(failed for rid)=======================================
Tr

Traceback (most recent call last):
  File "/home/ec2-user/SageMaker/snorkel_tagger/dataset_creator.py", line 42, in rememberv2_read
    results = json.loads(requests.post(url=url, data=json.dumps(payload), headers=headers).text)["Results"]
KeyError: 'Results'

==========================OBSERVATIONS OF SOME FALILURES(failed for rid)=======================================
Traceback (most recent call last):
  File "/home/ec2-user/SageMaker/snorkel_tagger/dataset_creator.py", line 119, in aggregate_formatted_entities
    recall_txn = rememberv2_read(docid)[0]
KeyError: 0

RID ====>  1051f852-bee3-4cce-82a6-df24ddafc180  
Traceback (most recent call last):
  File "/home/ec2-user/SageMaker/snorkel_tagger/dataset_creator.py", line 42, in rememberv2_read
    results = json.loads(requests.post(url=url, data=json.dumps(payload), headers=headers).text)["Results"]
KeyError: 'Results'

==========================OBSERVATIONS OF SOME FALILURES(failed for rid)=======================================
Tr

In [ ]:
trained_labels.head()

In [6]:
holder = pd.read_csv("combined_good_ocr.csv")

In [7]:
holder

,word_tokens,ocr,bounding_box,string_index,page_ind,rid,page
0,word_7_177,we,"[640, 1243, 660, 1287]",93,page_1,ea61cd23-d2cb-4ef7-8aba-64fb749eec5e,1
1,word_7_298,"Hmong,","[1117, 1598, 1150, 1716]",91,page_1,ea61cd23-d2cb-4ef7-8aba-64fb749eec5e,1
2,word_7_178,may,"[640, 1298, 667, 1363]",91,page_1,ea61cd23-d2cb-4ef7-8aba-64fb749eec5e,1
3,word_7_299,"Laotian,","[1117, 1730, 1150, 1851]",90,page_1,ea61cd23-d2cb-4ef7-8aba-64fb749eec5e,1
4,word_7_179,not,"[635, 1375, 661, 1422]",91,page_1,ea61cd23-d2cb-4ef7-8aba-64fb749eec5e,1
...,...,...,...,...,...,...,...
576340,word_499_274,X,"[1154, 155, 1184, 185]",73,page_1,e3b4f2e3-6441-4d9a-8b1c-2c0a0afc0c1a,1
576341,word_499_395,collected,"[1760, 646, 1793, 799]",91,page_1,e3b4f2e3-6441-4d9a-8b1c-2c0a0afc0c1a,1
576342,word_499_390,the,"[1760, 241, 1793, 295]",92,page_1,e3b4f2e3-6441-4d9a-8b1c-2c0a0afc0c1a,1
576343,word_499_270,O,"[1103, 1616, 1133, 1646]",52,page_1,e3b4f2e3-6441-4d9a-8b1c-2c0a0afc0c1a,1


In [5]:
h = get_snorkel_labels(,"AUTOMATED_UNDERWRITING_FEEDBACK_-_DU_CODIFIED_FINDINGS")

  0%|          | 1/576345 [00:00<20:37:42,  7.76it/s]

==============================Labeling is now started=======================================


AttributeError: 'Series' object has no attribute 'text'